In [76]:
!pip install torchaudio ffmpeg-python pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ffmpeg-python]


In [1]:
from pypdf import PdfReader

In [2]:
import os

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
from openai import OpenAI

In [6]:
import os

In [7]:
import chromadb

In [8]:
chroma_client = chromadb.PersistentClient(path = "/Users/Rakshit.Lodha/doc-q-a")

In [25]:
collection_1 = chroma_client.create_collection(name = "doc-qa-2")

In [27]:
collection_1.count()

0

In [28]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [29]:
import pandas as pd

In [30]:
import numpy as np

In [31]:
from tqdm import tqdm

In [84]:
import base64, io, wave

In [85]:
from IPython.display import Audio, display

In [86]:
from IPython.display import Javascript, display

In [90]:
import base64

In [93]:
from ipykernel.comm import Comm

In [32]:
pdf_file_path = "/Users/Rakshit.Lodha/doc-q-a/scan.pdf"

In [33]:
with open(pdf_file_path, 'rb') as pdf_file:
    reader = PdfReader("scan.pdf")
    number_of_pages = len(reader.pages)

In [34]:
text = " "

for page in reader.pages:
    text += page.extract_text() + '\n'

In [35]:
lines = text.split("\n")

lines

[' A Scandal in Bohemia',
 'Arthur Conan Doyle',
 'This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the',
 'text or any medium it may be on, including but not limited to warranties of merchantablity or ﬁtness for a particular purpose.',
 'This text was formatted from various free ASCII and HTML variants. See http:/ /sherlock-holm.es for an electronic form of this text',
 'and additional information about it.',
 'This text comes from the collection’s version 3.1.',
 'Table of contents',
 'Chapter 1. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .3',
 'Chapter 2. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .7',
 'Chapter 3. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .12',
 '1',
 '',
 'CHAPTE

In [36]:
processed_lines = []

i = 0

while i < len(lines):
    current_line = lines[i].strip()

    if current_line.endswith('-'):

        next_line = lines[i+1].strip()
        merged_line = current_line[:-1] + next_line

        processed_lines.append(merged_line)

        i +=2
    else:
        processed_lines.append(lines[i].strip())
        i +=1

processed_lines

['A Scandal in Bohemia',
 'Arthur Conan Doyle',
 'This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the',
 'text or any medium it may be on, including but not limited to warranties of merchantablity or ﬁtness for a particular purpose.',
 'This text was formatted from various free ASCII and HTML variants. See http:/ /sherlock-holm.es for an electronic form of this text',
 'and additional information about it.',
 'This text comes from the collection’s version 3.1.',
 'Table of contents',
 'Chapter 1. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .3',
 'Chapter 2. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .7',
 'Chapter 3. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .12',
 '1',
 '',
 'CHAPTER

In [37]:
paragraphs = []

current_paragraph = ""

for line in processed_lines:
    if line.endswith(('?','.', '!')):
        current_paragraph += line
        paragraphs.append(current_paragraph)
        current_paragraph = ""
    else:
        current_paragraph += line + " "

final_paragraphs = paragraphs[4:]

In [38]:
final_paragraphs

['T o Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer. They were admirable things for the observer—excellent for drawing the veil from men’s motives and actions. But for the trained reasoner to admit such intrusions into his own delicate and ﬁnely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results. Grit in a sensitive instrument, or a crack in one of his own high-power lenses, would not be more disturbing than a strong em

In [39]:
joined = []

char = ""

for para in final_paragraphs:
    if len(char) + len(para) <= 1000:
        char += para + " "
    elif len(char) + len(para)>1000:
        joined.append(char)
        char = para

if char:
    joined.append(char)

In [40]:
final_output = joined[1:]

In [41]:
len(joined)

54

In [42]:
def create_embeddings(text, batch_size = 50):
    all_embeddings = []

    for i in tqdm(range(0, len(text), batch_size), desc = "Creating Embeddings"):
        batch = text[i:i+batch_size]

        embeddings = client.embeddings.create(
            model = "text-embedding-3-small",
            input = batch
        )

        batch_embeddings = [item.embedding for item in embeddings.data]

        all_embeddings.extend(batch_embeddings)

    return all_embeddings

In [43]:
final_emb = create_embeddings(final_output)

Creating Embeddings: 100%|████████████████████████| 2/2 [00:05<00:00,  2.54s/it]


In [44]:
def id_creation(text):

    final_id = []

    for index, element in enumerate(text, start = 1):
        ids = f"id_{index}"
        final_id.append(ids)

    return final_id

In [45]:
id_created = id_creation(final_output)

In [46]:
def chroma_db_creation():

    batch_size = 50

    for i in tqdm(range(0, len(id_created), batch_size), desc = "Adding to Chroma"):
        batch_ids = id_created[i:i+batch_size]
        batch_embeddings = final_emb[i:i+batch_size]
        batch_documents = final_output[i:i+batch_size]

        collection_1.add(
            ids = batch_ids,
            embeddings = batch_embeddings,
            documents=batch_documents
        )

In [47]:
chroma_db_creation()

Adding to Chroma: 100%|███████████████████████████| 2/2 [00:00<00:00, 34.18it/s]


In [48]:
def query(text, results=3):

    query_conversion = client.embeddings.create(
        model = "text-embedding-3-small",
        input = text
    )

    result = collection_1.query(
        query_embeddings = query_conversion.data[0].embedding,
        n_results = results
    )

    return result['documents'][0]
    

In [49]:
query("Who is Irene?",5)

['T o Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer. They were admirable things for the observer—excellent for drawing the veil from men’s motives and actions. But for the trained reasoner to admit such intrusions into his own delicate and ﬁnely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results. Grit in a sensitive instrument, or a crack in one of his own high-power lenses, would not be more disturbing than a strong em

In [50]:
def query_answer(text):
    answers = query(text, 5)

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [
            {
                "role": "system", "content": f"""
                You need to answer the {text} based on the {answers} you receive. But before you answer, you need to make sure to re-rank
                the {answers} based on what you think is right and then use it to give a good answer which is short and conscise. 

                IMPORTANT RULE WHICH YOU SHOULD NOT FORGET:
                In case the {text} is something which is out of scope, and completely out of whack, answer with: I don't know about this
                EVEN if you get {answers}!
                """
            }
        ], temperature = 0.3
    )

    return response.choices[0].message.content

In [51]:
query_answer("Who is Irene?")

'Irene Adler is a character from the Sherlock Holmes stories by Arthur Conan Doyle. She is referred to by Holmes as "the woman," indicating her unique status in his life. Although Holmes does not have romantic feelings for her, he respects her intelligence and wit, as she is one of the few people to have outsmarted him. She is described as an adventuress and is involved in a case that threatened to cause a scandal in the kingdom of Bohemia.'

In [52]:
query_answer("Who is SRK?")

"I don't know about this."

In [53]:
query_answer("Give me the synopsis of this story")

"Sherlock Holmes, in disguise, investigates Irene Adler's home to retrieve a compromising photograph. He stages a street altercation, feigns injury, and is taken into Adler's home. Holmes uses a trick to discover the photograph's location, relying on Adler's instinct to protect her most valued possession. Despite feeling guilty about deceiving Adler, Holmes is committed to his plan to prevent her from using the photograph against his client."

In [54]:
query_answer("Did the end up retrieving the photograph of Irene?")

'Sherlock Holmes did not retrieve the photograph of Irene Adler. Although he found a photograph and a letter left for him by Irene, the photograph was not the one he was seeking to secure for the King. Irene Adler outsmarted Holmes by taking the original photograph with her when she left England with her husband, ensuring it would not be used against her.'

In [55]:
query_answer("What did the letter say?")

'The letter from Irene Adler to Sherlock Holmes stated that she and her husband had left and that the photograph, which was of concern to the King, was safe. She assured Holmes that the King could act without any hindrance from her, as she loved and was loved by a better man than the King. She left a photograph for the King to possess and signed the letter as Irene Norton, née Adler.'

In [126]:
audio_file_path = "/Users/Rakshit.Lodha/Downloads/recording (3).webm"

display(Audio(audio_file_path))

In [127]:
with open(audio_file_path, "rb") as audio_file:
    transcriptions = client.audio.transcriptions.create(
        model = "gpt-4o-transcribe",
        file = audio_file
    )

transcriptions.text

'Who is Sherlock Holmes?'

In [129]:
def audio_output():

    display(Javascript("""
    (async () => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const recorder = new MediaRecorder(stream);
    const chunks = [];
    
    recorder.ondataavailable = e => chunks.push(e.data);
    
    recorder.start();
    
    console.log("Recording...");
    await new Promise(r => setTimeout(r, 5000));
    recorder.stop();
    recorder.onstop = () => {
    const blob = new Blob(chunks, { type: recorder.mimeType });
    const url = URL.createObjectURL(blob);

    const a = document.createElement("a");
    a.href = url;
    a.download = "recording.webm";
    a.click();

    console.log("Recording saved as recording.webm");
    };
    })();
    """))

    audio_file_path = "/Users/Rakshit.Lodha/Downloads/recording (4).webm"

    return audio_file_path

In [148]:
def get_recordings():

    downloads = "/Users/Rakshit.Lodha/Downloads/"

    files = []
    
    for f in os.listdir(downloads):
        if f.startswith("recording-") and f.endswith(".webm"):
            file_path = os.path.join(downloads,f)
            files.append(file_path)
        else:
            f"No file found"

    with open(audio_file_path, "rb") as audio_file:
        transcriptions = client.audio.transcriptions.create(
            model = "gpt-4o-transcribe",
            file = audio_file
        )
    
    return transcriptions.text

In [149]:
get_recordings()

'Who is Sherlock Holmes?'